In [1]:
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np

In [2]:
df = pd.read_pickle("Ratings.pkl")

In [4]:
df.duplicated(['name','review']).sum()

409

In [5]:
df.drop_duplicates(['name','review'],inplace = True)

In [6]:
df= df.reset_index(drop=True)

In [7]:
df.shape

(130618, 10)

In [8]:
df.rate_pred.value_counts()

 1    83152
-1    29359
 0    18107
Name: rate_pred, dtype: int64

In [9]:
def rating(rating):
    if(rating > 3):
        return 1
    else:
        return 0
    
df['rate_pred1'] = df['rating'].apply(rating) 
df.head()

,name,rating,review,Lemma,Subjectivity,Polarity,Analysis,Vader Sentiment,Vader Analysis,rate_pred,rate_pred1
0,Jalsa,4.0,a beautiful place to dine inthe interiors take...,beautiful place dine inthe interior take back ...,0.438624,0.229762,1,0.7351,1,1,1
1,Jalsa,4.0,i was here for dinner with my family on a week...,dinner family weekday restaurant completely em...,0.533333,0.500000,1,0.9646,1,1,1
2,Jalsa,2.0,its a restaurant near to banashankari bda me a...,restaurant near banashankari bda along office ...,0.566667,0.183333,1,0.7845,1,-1,0
3,Jalsa,4.0,we went here on a weekend and one of us had th...,went weekend one u buffet two u took ala carte...,0.655556,0.641667,1,0.9686,1,1,1
4,Jalsa,5.0,the best thing about the place is its ambiance...,best thing place ambiance second best thing yu...,0.368750,0.340625,1,0.9485,1,1,1


In [10]:
df.rate_pred1.value_counts()

1    83152
0    47466
Name: rate_pred1, dtype: int64

In [11]:
#random split train and test data
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.70]
test = df[df['random_number'] > 0.70]

In [12]:
train.shape

(99115, 12)

### Applying CountVectorizer

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b',ngram_range=(1,2))
train_matrix = vectorizer.fit_transform(train['Lemma'])
test_matrix = vectorizer.transform(test['Lemma'])
print(train_matrix)

  (0, 73443)	1
  (0, 678125)	1
  (0, 246993)	1
  (0, 447795)	1
  (0, 447064)	1
  (0, 888301)	1
  (0, 57953)	1
  (0, 573238)	1
  (0, 286853)	1
  (0, 495611)	1
  (0, 667666)	1
  (0, 995514)	1
  (0, 606248)	1
  (0, 163203)	1
  (0, 501447)	1
  (0, 450682)	1
  (0, 50147)	1
  (0, 892591)	1
  (0, 803371)	1
  (0, 185219)	1
  (0, 14226)	1
  (0, 183126)	1
  (0, 107536)	1
  (0, 197254)	1
  (0, 84158)	1
  :	:
  (99114, 196107)	1
  (99114, 170371)	1
  (99114, 707206)	1
  (99114, 627381)	1
  (99114, 458418)	1
  (99114, 686640)	1
  (99114, 18795)	1
  (99114, 84975)	1
  (99114, 827481)	1
  (99114, 649556)	1
  (99114, 502499)	1
  (99114, 565753)	1
  (99114, 276052)	1
  (99114, 670191)	1
  (99114, 374593)	1
  (99114, 84996)	1
  (99114, 750394)	1
  (99114, 614368)	1
  (99114, 670192)	1
  (99114, 649562)	1
  (99114, 757527)	1
  (99114, 538428)	1
  (99114, 458446)	1
  (99114, 541125)	1
  (99114, 170458)	1


### Logistic Regression

In [63]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [64]:
X_train = train_matrix
X_test = test_matrix
y_train = train['rate_pred1']
y_test = test['rate_pred1']

In [47]:
lr.fit(X_train,y_train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Saving the model

In [65]:
import pickle
filename = 'LogisticRegression.pkl' 
with open(filename,'wb') as file:
    pickle.dump(lr , file)


In [67]:
with open(filename,'rb') as file:
    loaded_model = pickle.load(file)
loaded_model


LogisticRegression()

In [50]:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[ 9101,  1724],
       [ 2355, 18323]], dtype=int64)

In [51]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.79      0.84      0.82     10825
           1       0.91      0.89      0.90     20678

    accuracy                           0.87     31503
   macro avg       0.85      0.86      0.86     31503
weighted avg       0.87      0.87      0.87     31503



### Accuracy Obtained from Logistic Regression 0.87 

### MultinomialNB 

In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import  MultinomialNB
nb_clf = MultinomialNB()
from sklearn.metrics import classification_report
parameters = {  
'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
}  
clf = GridSearchCV(nb_clf, parameters, cv=10, scoring='neg_root_mean_squared_error')
grid_result=clf.fit(X_train, y_train)

In [70]:
print(grid_result.best_score_,grid_result.best_params_)

-0.37764526337683735 {'alpha': 1}


In [71]:
import pickle
filename = 'MultinomialNB.sav'
pickle.dump(clf , open(filename,'wb'))
##load the model
loaded_model = pickle.load(open(filename,'rb'))
result = loaded_model.score(X_test,y_test)


In [72]:
print(result)
print(classification_report(y_test, grid_result.predict(X_test), digits=4))

-0.3808747160324033
              precision    recall  f1-score   support

           0     0.9045    0.6720    0.7711     11456
           1     0.8366    0.9594    0.8938     20047

    accuracy                         0.8549     31503
   macro avg     0.8705    0.8157    0.8325     31503
weighted avg     0.8613    0.8549    0.8492     31503



In [25]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(max_depth=3)
decision_tree = decision_tree.fit(X_train,y_train)
print(classification_report(y_test, decision_tree.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     0.8962    0.2117    0.3425     11456
           1     0.6864    0.9860    0.8094     20047

    accuracy                         0.7044     31503
   macro avg     0.7913    0.5988    0.5759     31503
weighted avg     0.7627    0.7044    0.6396     31503



### Accuracy Obtained from MultinomialNB 0.70

### Random Forest Classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth = 3).fit(X_train, y_train)


In [27]:
print(classification_report(y_test, classifier.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000     11456
           1     0.6364    1.0000    0.7778     20047

    accuracy                         0.6364     31503
   macro avg     0.3182    0.5000    0.3889     31503
weighted avg     0.4049    0.6364    0.4949     31503



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Accuracy Obtained from Random Forest Classifier 0.63

### Adaboost Classifier

In [28]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
abc = abc.fit(X_train,y_train)
print(classification_report(y_test, abc.predict(X_test), digits=4))

              precision    recall  f1-score   support

           0     0.7727    0.7382    0.7551     11456
           1     0.8541    0.8759    0.8649     20047

    accuracy                         0.8258     31503
   macro avg     0.8134    0.8071    0.8100     31503
weighted avg     0.8245    0.8258    0.8249     31503



### Accuracy Obtained from Adaboost Classifier 0.82

### Long-Short Term Memory 

In [33]:
!pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=971c35eed0ed955f500877f1b0acfe989cbfe07aa31cb503aaca7fefd9d69adc
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor


In [78]:

from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [79]:
x_train, x_test, y_train, y_test = train_test_split(df['Lemma'], df['rate_pred1'], test_size = 0.2)
#print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

93026     hi first experience smoke ben highly recommend...
37663     ancient place nostalgia strike every time go l...
27868     one worst experience ever waiter deny one orde...
73298     wanted try waffle sandwich chose sin waffle bw...
55761     one favorite place dine dahi ke kebab die grea...
                                ...                        
30536     good food great service quality product custom...
57279     celebrate birthday place really nice polite st...
120629                 received rubber band gobi manchurian
73057     even thought delivered late im giving four sta...
79609     one good outlet behrouz biryani best biryani s...
Name: Lemma, Length: 104494, dtype: object 

109406    visited many place ab barbeque nation many bbq...
44060     ordered konaseema chicken starter aandhrautton...
42166                                     good quality food
51481                                         late delivery
6328      ordered paneer bhurji poha packing really nic

In [80]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [81]:

# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[1485   98   33 ... 1165    1    0]
 [7333    3 3760 ...   21    0    0]
 [   8  119   33 ...    0    0    0]
 ...
 [ 491 2220 2668 ...    0    0    0]
 [  43  408  141 ...    0    0    0]
 [   8    2  121 ...    0    0    0]] 

Encoded X Test
 [[ 118  138    3 ...   32  254 1788]
 [   7 4437    4 ...    0    0    0]
 [   2   27    1 ...    0    0    0]
 ...
 [   8  168   56 ...    0    0    0]
 [  50  277    0 ...    0    0    0]
 [   3   18 1107 ...    0    0    0]] 

Maximum review length:  29


In [82]:
print(type(x_train))

<class 'numpy.ndarray'>


In [83]:
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 29, 32)            2347008   
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,371,905
Trainable params: 2,371,905
Non-trainable params: 0
_________________________________________________________________
None


In [84]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [85]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
817/817 [==============================] - ETA: 0s - loss: 0.3717 - accuracy: 0.8364
Epoch 00001: accuracy improved from -inf to 0.83635, saving model to models\LSTM.h5
817/817 [==============================] - 52s 62ms/step - loss: 0.3717 - accuracy: 0.8364
Epoch 2/5
817/817 [==============================] - ETA: 0s - loss: 0.2914 - accuracy: 0.8776
Epoch 00002: accuracy improved from 0.83635 to 0.87765, saving model to models\LSTM.h5
817/817 [==============================] - 52s 64ms/step - loss: 0.2914 - accuracy: 0.8776
Epoch 3/5
816/817 [============================>.] - ETA: 0s - loss: 0.2542 - accuracy: 0.8926
Epoch 00003: accuracy improved from 0.87765 to 0.89261, saving model to models\LSTM.h5
817/817 [==============================] - 52s 63ms/step - loss: 0.2542 - accuracy: 0.8926
Epoch 4/5
816/817 [============================>.] - ETA: 0s - loss: 0.2224 - accuracy: 0.9052
Epoch 00004: accuracy improved from 0.89261 to 0.90517, saving model to models\LSTM.h5
81

In [86]:
y_pred = model.predict(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
#    print(round(y_pred[i][0]))
     if y == (round(y_pred[i][0])):
         true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

Correct Prediction: 22067
Wrong Prediction: 4057
Accuracy: 84.4702189557495


### Accuracy Obtained from LSTM 0.84

In [87]:
model.evaluate(x_test,y_test)


817/817 [==============================] - 8s 9ms/step - loss: 0.4515 - accuracy: 0.8447


[0.4515098035335541, 0.8447021842002869]

### LSTM model,logistic regression, multinomialNB performs better than others